In [1]:
import os
import re
import pandas as pd
import numpy as np
import tqdm
import ast

In [2]:
%pip install openai
import openai

Note: you may need to restart the kernel to use updated packages.


In [8]:
openai.api_type = "azure"
# openai.azure_endpoint = 
# openai.api_version = "2023-03-15-preview"
# openai.api_key = 


import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key= 
    api_version="2023-10-01-preview",
    azure_endpoint = 
)

In [9]:
def getExplanation(prompt,prefix=None):

    if(prefix is not None):
        default = prefix
    else:
        default = '''You are given a mathematical problem and its answer. Show the step by step working to start from the problem and obtain the solution.
        Example: 
        Problem: I ate 65 oranges, followed by 5 apples. How many fruits did I eat?
        Answer: 70
        Solution steps: In order to find the total number of fruits eaten, we add the number of oranges (65) and apples (5). 65 and 5 make 70. FINAL ANSWER: 70

        Example:
        Problem: 

        Now do the same for the following problem and answer.
        {}
        Solution steps: 
        '''.format(prompt)


    return default


In [13]:
def createOP(prompt,prefix=None):

    if(prefix is not None):
        default = prefix
    else:
        default = '''
        You are given an input json string to parse into different categories.
        From the following text, output the following.
        Context: The part of the input that contains the value of "augmented passage" 
        Problem: The part of the input that occurs after "question:"
        Relevant variables: The part of the input that occurs before "augmented passage"
        Irrelevant variables: The part of the output containing value of "new variables". If this is not present in the input, do not include this key in the output.
        Explanation: An explanation of why the irrelevant variables are not related to the problem. If irrelevant (new) variables are not in the input, do not include this key in the output
        Solution: The part of the input containing the "Solution Steps" string.

        Example:
        Input: [['Number of pencils (changes from 2 to 3)', 'Augmented: There are 2 pencils in the drawer with a weight of 10 grams each. Tim placed 3 pencils, each weighing 12 grams, in the drawer. Now the pencils in the drawer have a total weight of 56 grams.', 'New Variables: Weight (changes from 10 to 12)']]. Question: What is the total number of pencils? Solution steps: Initially there were 2 pencils. 3 more were added. Hence the total number of pencils is the sum of 2 and 3, which is 5. FINAL ANSWER: 5
        Output: 
        Context: There are 2 pencils in the drawer with a weight of 10 grams each. Tim placed 3 pencils, each weighing 12 grams, in the drawer. Now there are 5 pencils in the drawer with a total weight of 58 grams.
        Problem: What is the total number of pencils?
        Relevant variables: Number of pencils (changes from 2 to 5)
        Irrelevant variables: Weight (changes from 10 to 12)
        Explanation: As the problem deals with number of pencils, which we already have, there is no need to consider the weight of the pencils when counting the total quantity.
        Solution: Initially there were 2 pencils. 3 more were added. Hence the total number of pencils is the sum of 2 and 3, which is 5. FINAL ANSWER: 5

        Now do the same for the following.
        Input: {}
        Output:        
        '''.format(prompt)





    return default

In [11]:

def getInference(prompt_text, returnPrompt=False):

    # print(prompt_text)
    # print(type(prompt_text))

    response = openai.chat.completions.create(
    # response = client.completions.create(
        model="gpt-35-turbo",
        messages = [

                        {"role": "system", "content": "You are a mathematician."},

                        {"role": "user", "content": prompt_text}

                    ],
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)

    ans = response.choices[0].message.content

    if(returnPrompt):
        return [prompt_text,ans[0]]
    else:
        return ans 

In [18]:
for i in tqdm.tqdm(range(0,10)):

    
    # def process(ipfile,opfile)
    ipfile = f'SIMPLE-stage_4_with_adversarial_TRAIN_SHARD_{i}.csv'
    opfile = os.path.join('updated',ipfile)
    df = pd.read_csv(ipfile)
    df['unique_id'] = df['passage_id']+'_'+df['query_id']
    df['answers'] = df['answer'].apply(lambda x: ast.literal_eval(x)['number'])
    # df.info()
    nonadv = df[~df.isAdversarial]
    adv = df[df.isAdversarial]

    nonadv['ip'] = 'Problem: ' + nonadv['passage'] + '.' + nonadv['question'] + '\nAnswer: ' + nonadv['answers']
    print(nonadv['ip'].iloc[0])

    nonadv['output'] = nonadv['ip'].apply(lambda x: getInference(getExplanation(x)))
    nonadv['input'] = nonadv['ip'].str.split('\nAnswer').str[0]

    nonadv.to_csv(os.path.join('explanations',ipfile))

    print('non adv done. we have solutions')

    adv['output'] = nonadv['output']
    adv['ip'] = adv['passage'] + 'Question: ' + adv['question'] + 'Solution steps: ' + adv['output']

    adv['output'] = adv['ip'].apply(lambda x: getInference(createOP(x)))

    print('adv done. we have explanations.')

    nonadv.to_csv(os.path.join('explanations',ipfile.replace('.csv','_ADV.csv')))


    try:
            
        adv['input'] = adv['passage'].apply(lambda x: ast.literal_eval(x)[0][1]).str.split('Augmented: ').str[-1]
        adv2 = adv[['input','output']]
        nonadv2 = nonadv[['input','output']]
        dfo = pd.concat([nonadv2,adv2])
        dfo = dfo.reset_index(drop=True)
        dfo.to_csv(opfile,index=False)
    except: 
        continue


  0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_3344224/3006063650.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonadv['ip'] = 'Problem: ' + nonadv['passage'] + '.' + nonadv['question'] + '\nAnswer: ' + nonadv['answers']
/tmp/ipykernel_3344224/3006063650.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonadv['output'] = nonadv['ip'].apply(lambda x: getInference(getExplanation(x)))
/tmp/ipykernel_3344224/3006063650.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

Problem: A 200 seat theater charges 3.00 dollars for adults and 1.50 dollars for children. If all seats were filled and the total ticket income was 510 dollars how many adults and children were in the audience? 
Answer: 140,60
non adv done. we have solutions
adv done. we have explanations.
Problem: The state market has 27 more apples than oranges. There are 301 apples and oranges all together. How many apples are in the market?
Answer: 164
non adv done. we have solutions
adv done. we have explanations.
Problem: An investor will invest a total of 15000 dollars in 2 accounts , one paying 4 % annual simple interest and the other 3 %. If he wants to earn 550 dollars annual interest , how much should he invest at 4 %? How much at 3 %? 
Answer: 10000,5000
non adv done. we have solutions
adv done. we have explanations.
Problem: There are 3 more girls than boys in Tom 's class , and the sum of girls and boys is 41. How many boys are in the class? How many girls are in the class? 
Answer: 19,22

In [ ]:
# adv.head()

In [ ]:
# nonadv.head()

In [ ]:
# adv.head()

In [ ]:
### now get the input column.

In [ ]:
### finally, keep only input and output columns.

In [ ]:
## algo:


## if non-adv example, pass passage+question+answer to getExplanation.
## if adv example, pass to createOP with augmented passage + new vars + augmented vars.
## receive this output and attach the getExplanation output at the end

        ## for non-adv
        # we kep context, proble, solution.
        ## for adv:
        # we keep all